In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import joblib
import warnings

warnings.filterwarnings('ignore')

In [2]:

# Veri yükleme ve ön işleme
data = pd.read_csv('Amazon.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.set_index('Date')['Close']

In [3]:
# NaN ve sonsuz değerleri kontrol etme ve temizleme
data = data.replace([np.inf, -np.inf], np.nan).dropna()

In [4]:
# Durağanlık testi
def test_stationarity(timeseries):
    result = adfuller(timeseries, autolag='AIC')
    print('ADF İstatistiği:', result[0])
    print('p-değeri:', result[1])
    print('Kritik Değerler:', result[4])
    if result[1] <= 0.05:
        print("Seri durağandır (H0 reddedilir)")
    else:
        print("Seri durağan değildir (H0 reddedilemez)")

In [5]:
print("Orijinal serinin durağanlık testi:")
test_stationarity(data)

Orijinal serinin durağanlık testi:
ADF İstatistiği: 5.618546547149937
p-değeri: 1.0
Kritik Değerler: {'1%': -3.431476604615945, '5%': -2.8620378526446375, '10%': -2.567035004946881}
Seri durağan değildir (H0 reddedilemez)


In [6]:
# Log dönüşümü ve fark alma ile seriyi durağan hale getirme
data_log = np.log(data)
data_diff = data_log.diff().dropna()
print("\nFark alınmış log serisinin durağanlık testi:")
test_stationarity(data_diff)


Fark alınmış log serisinin durağanlık testi:
ADF İstatistiği: -13.017840358325302
p-değeri: 2.4894500683768166e-24
Kritik Değerler: {'1%': -3.431476604615945, '5%': -2.8620378526446375, '10%': -2.567035004946881}
Seri durağandır (H0 reddedilir)


In [7]:
# Veriyi eğitim ve test setlerine ayırma
train_size = int(len(data_diff) * 0.8)
train, test = data_diff[:train_size], data_diff[train_size:]


In [8]:
# ARIMA modelini optimize etme
def optimize_arima(train, test):
    best_score, best_cfg = float("inf"), None
    p_values = range(0, 7)
    d_values = [0, 1]
    q_values = range(0, 7)

    tscv = TimeSeriesSplit(n_splits=5)

    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    cv_scores = []
                    for train_index, val_index in tscv.split(train):
                        cv_train, cv_val = train.iloc[train_index], train.iloc[val_index]
                        model = ARIMA(cv_train, order=order)
                        results = model.fit()
                        forecast = results.forecast(steps=len(cv_val))
                        mse = mean_squared_error(cv_val, forecast)
                        cv_scores.append(mse)

                    avg_mse = np.mean(cv_scores)
                    if avg_mse < best_score:
                        best_score, best_cfg = avg_mse, order
                        print(f"Yeni en iyi konfigürasyon: {order}, MSE: {avg_mse}")
                except:
                    continue
    return best_cfg

In [9]:
best_order = optimize_arima(train, test)
print(f"En iyi ARIMA düzeni: {best_order}")

Yeni en iyi konfigürasyon: (0, 0, 0), MSE: 0.0011793369648792442
Yeni en iyi konfigürasyon: (0, 0, 1), MSE: 0.0011793366338401629
Yeni en iyi konfigürasyon: (0, 1, 1), MSE: 0.001177436367790475
Yeni en iyi konfigürasyon: (0, 1, 2), MSE: 0.0011773954086554714
Yeni en iyi konfigürasyon: (0, 1, 6), MSE: 0.001176841673657287
Yeni en iyi konfigürasyon: (4, 1, 3), MSE: 0.0011768109368170108
En iyi ARIMA düzeni: (4, 1, 3)


In [32]:
# En iyi ARIMA modelini eğitme
model = ARIMA(train, order=best_order)
results = model.fit()

In [33]:
# Tahmin yapma ve RMSE hesaplama
forecast = results.forecast(steps=len(test))
rmse = np.sqrt(mean_squared_error(test, forecast))
print(f'ARIMA RMSE: {rmse}')

ARIMA RMSE: 0.01883234831863641


In [34]:
# Tahminleri orijinal ölçeğe geri dönüştürme
forecast_cumsum = forecast.cumsum()
last_train_value = data_log.iloc[train_size - 1]
forecast_log = pd.Series(last_train_value) + forecast_cumsum
forecast_original_scale = np.exp(forecast_log)

In [41]:
# Save the ARIMA model
model_filename = 'arima_model.joblib'
joblib.dump(results, model_filename)
print(f"ARIMA model saved as {model_filename}")

ARIMA model saved as arima_model.joblib


In [42]:
# Save the last observation of the training data
last_observation = data.iloc[train_size - 1]
observation_filename = 'last_observation.joblib'
joblib.dump(last_observation, observation_filename)
print(f"Last observation saved as {observation_filename}")

Last observation saved as last_observation.joblib


In [43]:
# Save the best order
order_filename = 'best_order.joblib'
joblib.dump(best_order, order_filename)
print(f"Best order saved as {order_filename}")

Best order saved as best_order.joblib
